# Setup 

* Installs necessary packages 

* Downloads data to local storage

In [ ]:
!pip install transformers
!rm -r *
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!rm -r annotations_trainval2017.zip
!rm -r annotations/instances_train2017.json
!rm -r annotations/instances_val2017.json
!rm -r annotations/person_keypoints_train2017.json
!rm -r annotations/person_keypoints_val2017.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 31.3 MB/s 
     |████████████████████████████████| 6.6 MB 54.4 MB/s 
     |████████████████████████████████| 120 kB 74.1 MB/s 
--2022-08-24 11:05:54--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.135.73
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.135.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  12.4MB/s    in 22s     

2022-08-24 11:06:16 (11.2 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: 

# Imports

* Importation of necessary packages for the following code to work.

In [ ]:
import json
import time
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import logging
import matplotlib.pyplot as plt
from PIL import Image
import requests
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    ViTFeatureExtractor,
    BertTokenizer,
)
logging.set_verbosity_error()

# Configuration 
* Choice of Image & Text encoders
* Choice of learning params


In [ ]:
class CFG:

    max_text_tokens_length = 128
    text_backbone = 'bert-base-uncased'
    image_backbone = 'google/vit-base-patch16-224'

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_size = 32
    max_epochs = 75
    max_bad_epochs = 9
    patience = 3
    factor = 0.1

# Functions

* Definition of COCO dataset loading methods
* Definition of Learning methods 

In [ ]:
def read_coco_pairs(split='train'):
  if split == 'train':
    path = 'annotations/captions_train2017.json'
  else:
    path = 'annotations/captions_val2017.json'
  with open(path, 'r') as file:
      data = json.load(file)
  images = list(map(lambda x: (x['id'], x['flickr_url']), data['images']))
  images = {x[0]: x[1] for x in images}
  annotations = list(map(lambda x:(x['image_id'], x['caption']),data['annotations']))
  pairs = list(map(lambda x:{'caption':x[1],'url':images[x[0]]}, annotations))
  return pairs

class DataSet(torch.utils.data.Dataset):

  def __init__(self, pairs, processor):
    super().__init__()
    self.pairs = pairs
    self.processor = processor
  
  def __getitem__(self, idx):
    try:
      caption = self.pairs[idx]['caption']
      url = self.pairs[idx]['url']
      image = Image.open(requests.get(url, stream=True).raw) 
      encoded_pair = self.processor(text=[caption], images=[image], return_tensors="pt", max_length=CFG.max_text_tokens_length, padding='max_length', truncation=True)
      return encoded_pair
    except:
      return None
  
  def __len__(self):
    return len(self.pairs)

def collate_fn(batch):
  batch = list(filter(lambda x: x is not None, batch))
  return torch.utils.data.dataloader.default_collate(batch)

def train_epoch(model, train_loader, optimizer, epoch, max_epochs):
    model.train()
    nb_batches = len(train_loader)
    tqdm_object = tqdm(train_loader, total=len(train_loader))   
    epoch_loss = 0.0
    for i, batch in enumerate(tqdm_object):
      outputs = model(
          input_ids=batch['input_ids'].squeeze().to(CFG.device),
          attention_mask=batch['attention_mask'].squeeze().to(CFG.device),
          pixel_values=batch['pixel_values'].squeeze().to(CFG.device),
          return_loss=True)
      loss, logits_per_image = outputs.loss, outputs.logits_per_image  # this is the image-text similarity score
      loss.backward()
      optimizer.step()
      tqdm_object.set_postfix(
          batch="{}/{}".format(i+1,nb_batches),
          train_loss=loss.item(),
          lr=get_lr(optimizer)
          )
    epoch_loss = epoch_loss / nb_batches
    return epoch_loss

def valid_epoch(model, dev_loader, epoch, max_epochs):
    model.eval()
    nb_batches = len(dev_loader)
    tqdm_object = tqdm(dev_loader, total=len(dev_loader))
    epoch_loss = 0.0   
    for i, batch in enumerate(tqdm_object):
      outputs = model(
          input_ids=batch['input_ids'].squeeze(),
          attention_mask=batch['attention_mask'].squeeze(),
          pixel_values=batch['pixel_values'].squeeze(),
          return_loss=True)
      loss, logits_per_image = outputs.loss, outputs.logits_per_image  # this is the image-text similarity score
      epoch_loss += loss.item()
      tqdm_object.set_postfix(
          batch="{}/{}".format(i+1,nb_batches),
          dev_loss=loss.item(),
          )
    epoch_loss = epoch_loss / nb_batches
    return epoch_loss

def learning_loop(model):
    model.to(CFG.device)
    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=0.)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=CFG.patience, factor=CFG.factor)

    best_dev_score = float('inf')
    train_history = []
    dev_history = []
    nb_bad_epochs = 0

    print("Learning phase")
    print('Used device:', CFG.device)
    print("--------------")
    for epoch in range(1, CFG.max_epochs+1):

        print("Epoch {:03d}/{:03d}".format(epoch, CFG.max_epochs))

        if nb_bad_epochs >= CFG.max_bad_epochs:
            print("Epoch {:03d}/{:03d}: exiting training after too many bad epochs.".format(epoch, CFG.max_epochs))
            torch.save(model.state_dict(), "final.pt")
            break

        else:

            epoch_start_time = time.time()

            epoch_train_loss = train_epoch(model=model, train_loader=train_dataloader, optimizer=optimizer, epoch=epoch, max_epochs=CFG.max_epochs)
            epoch_dev_score = valid_epoch(model=model, dev_loader=val_dataloader, epoch=epoch, max_epochs=CFG.max_epochs)

            duration = time.time() - epoch_start_time

            lr_scheduler.step(epoch_dev_score)

            train_history.append(epoch_train_loss)
            dev_history.append(epoch_dev_score)

            if epoch_dev_score < best_dev_score:
                nb_bad_epochs = 0
                best_dev_score = epoch_dev_score
                torch.save(model.state_dict(), "best.pt")
                print("Finished epoch {:03d}/{:03d} - Train loss: {:.7f} - Valid loss: {:.7f} - SAVED (NEW) BEST MODEL. Duration: {:.3f} s".format(
                epoch, CFG.max_epochs, epoch_train_loss, epoch_dev_score, duration))
            else:
                nb_bad_epochs += 1
                print("Finished epoch {:03d}/{:03d} - Train loss: {:.7f} - Valid loss: {:.7f} - NUMBER OF BAD EPOCH.S: {}. Duration: {:.3f} s".format(
                epoch, CFG.max_epochs, epoch_train_loss, epoch_dev_score, nb_bad_epochs, duration))
    
    history = {'train':train_history,'dev':dev_history}
    return history
  
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

def plot_history(history):
    train_history = history['train']
    dev_history = history['dev']
    plt.plot(list(range(1, len(train_history)+1)), train_history, label="train loss")
    plt.plot(list(range(1, len(dev_history)+1)), dev_history, label="dev loss")
    plt.xticks(list(range(1, len(train_history)+1)))
    plt.xlabel("epoch")
    plt.legend()

# Initializing a data processor

* Init. of a text Tokenizer
* Init. of an image Feature extractor

In [ ]:
%%time
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
processor = VisionTextDualEncoderProcessor(feature_extractor, tokenizer)

CPU times: user 500 ms, sys: 38 ms, total: 538 ms
Wall time: 11.4 s


# Creating torch dataset

* Loading COCO (text, image) pairs  
* Wrapping pairs into a torch dataset, with the data processor attached 

In [ ]:
%%time
train_pairs = read_coco_pairs(split='train')
val_pairs = read_coco_pairs(split='val')
train_ds = DataSet(pairs=train_pairs, processor=processor)
dev_ds = DataSet(pairs=val_pairs, processor=processor)
train_dataloader = torch.utils.data.DataLoader(train_ds, collate_fn=collate_fn, batch_size=CFG.batch_size)
val_dataloader = torch.utils.data.DataLoader(dev_ds, collate_fn=collate_fn, batch_size=CFG.batch_size)

CPU times: user 2.12 s, sys: 437 ms, total: 2.56 s
Wall time: 3.46 s


# Initializing CLIP model 

* Instantiating a new CLIP model, using prescribed text & image pre-trained backbones.

In [ ]:
%%time
clip = VisionTextDualEncoderModel.from_vision_text_pretrained(CFG.image_backbone, CFG.text_backbone)

# Learning (fine-tuning) loop

* Updating weights of pre-trained encoders and projection head while minimizing constrastive loss.

In [ ]:
%%time
history = learning_loop(clip)

Learning phase
--------------
Used device: cuda
Epoch 001/075


 13%|█▎        | 2441/18493 [3:29:54<19:26:38,  4.36s/it, batch=2441/18493, lr=0.001, train_loss=3.18]

# Learning history

* Visualizing learning curves

In [ ]:
%%time
plot_history(history)

# Testing